In [1]:
import xml.dom.minidom as dom
import xml.sax as sax
import os

In [20]:
path = os.path.join(os.getcwd(), "CysticFibrosis2-20230406\data\cf79.xml")

In [21]:
### LENDO O ARQUIVO XML E GERANDO UM NOVO XML COM OS AUTORES UTILIZANDO A BIBLIOTECA DOM ###

# Leitura do arquivo XML
doc = dom.parse(path)
autores = doc.getElementsByTagName('AUTHOR')

# Criação de um novo arquivo XML
novo_doc = dom.Document()
autores_elemento = novo_doc.createElement('AUTHORS')
novo_doc.appendChild(autores_elemento)

# Adiciona cada autor em uma nova linha e garantindo que não haja repetição
nomes = []
for autor in autores:
    
    nome = autor.firstChild.data
    if nome not in nomes:
        
        nomes.append(nome)
    
        autor_elemento = novo_doc.createElement('AUTHOR')
        autor_nome = novo_doc.createTextNode(nome)
        autor_elemento.appendChild(autor_nome)

        quebra_linha = novo_doc.createTextNode('\n')
        autores_elemento.appendChild(quebra_linha)

        autores_elemento.appendChild(autor_elemento)

# Gravação do novo arquivo XML com os autores em linhas separadas
with open('autor.xml', 'w') as arquivo:
    arquivo.write(novo_doc.toprettyxml(indent='    '))

In [22]:
### LENDO O ARQUIVO XML E GERANDO UM NOVO XML COM OS TITULOS UTILIZANDO A BIBLIOTECA SAX ###

# Criando a classe que escreve cada titulo individualmente a partir da leitura do arquivo xml pela biblioteca SAX
class TitleContentHandler(sax.ContentHandler):
    def __init__(self, output_file):
        sax.ContentHandler.__init__(self)
        self.output_file = output_file
        self.in_title = False

    def startElement(self, name, attrs):
        if name == "TITLE":
            self.in_title = True
            self.output_file.write("<TITLE>")

    def characters(self, content):
        if self.in_title:
            self.output_file.write(sax.saxutils.escape(content.strip()))

    def endElement(self, name):
        if name == "TITLE":
            self.in_title = False
            self.output_file.write("</TITLE>\n")

# Gravação do novo arquivo XML com os titulos em linhas separadas
output_file = open("titulo.xml", "w")
output_file.write("<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n")
output_file.write("<TITLES>\n")
handler = TitleContentHandler(output_file)
sax.parse(path, handler)
output_file.write("</TITLES>")
output_file.close()